In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import os

In [ ]:
# file_names = !ls ../tx *.json
# file_names = list(file_names)
# path_to_tx = '../tx/'
path_to_tx = './tx_10_q/'

In [ ]:
# file_names = [file_name for file_name in file_names if '.json' in file_name]
# tx_files = [pos_json for pos_json in os.listdir(path_to_tx) if pos_json.endswith('.json')]
tx_files = [pos_json for pos_json in os.listdir(path_to_tx) if pos_json.endswith('.json')]

In [ ]:
len(tx_files)

In [ ]:
datas = []
count = 0
# for file_name in file_names:
#     with open('../tx/' + file_name) as f:
#         datas.append(json.load(f))
#     print(count)
#     count += 1

for tx in os.listdir(path_to_tx):
    with open('./tx_10_q//' + tx) as f:
        datas.append(json.load(f))
    print(count)
    count +=1

In [ ]:
# delete broken data in datas
datas = [data for data in datas if data != False]

In [ ]:
txs = pd.DataFrame.from_dict(json_normalize(datas), orient='columns')    

In [ ]:
# Testing session
# type(txs.iloc[6725]['vin']['txid'])
# print(SimplifyVout(txs.iloc[6725]))
# SimplifyVout(txs.iloc[6725])

In [ ]:
def SimplifyVin(tx):
    SimpVin = []
    for toward in tx['vin']:
        if 'txid' in toward.keys():
            txid = toward['txid']
            vout = toward['vout']
        else:
            txid = 'n/a'
            vout = -1
        SimpVin.append({'txid':txid,'vout':int(vout)})
    return SimpVin

In [ ]:
def SimplifyVout(tx):
    SimpVout = []
    for out in tx['vout']:
        
        if 'addresses' in out['scriptPubKey'].keys():
            value = out['value']
            address = out['scriptPubKey']['addresses']
#             print(out)
        else:
            value = -1
            address = ['n/a']
        SimpVout.append({'addresses':address,'value':float(value)})    
    return SimpVout

In [ ]:
TxBucket = []

for index, tx in txs.iterrows(): 
    
#     extract information from Vin
    SimpVin = SimplifyVin(tx)
#     extract information from Vout
    SimpVout = SimplifyVout(tx)
#     extract time and txid
    SimpTime = tx['time']
    SimpTxId = tx['txid']
    SimpTx = {'time':int(SimpTime), 'txid':SimpTxId,'vin':SimpVin, 'vout': SimpVout}
    TxBucket.append(SimpTx)
    
    print(index)

In [ ]:
# import csv
# with open('../DataCleaning/tx_13_output_Q.csv', 'w', newline='') as csvfile:
#   writer = csv.writer(csvfile)

#   # 寫入二維表格
#   writer.writerows(data_to_write)

In [ ]:
# output dictionary as json file
with open('./tx_N_output_.json', 'w') as fp:
    json.dump({'transactions':TxBucket}, fp)